# Desafio Bulk Spark Stream

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import shutil

# Criando SparkSession com conector mongo-spark
spark = SparkSession \
    .builder \
    .appName("bulk_stream_spark")\
    .config("spark.logConf", "true")\
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.0')\
    .getOrCreate()

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.conf.set('spark.sql.caseSensitive', True)

22/11/01 12:29:00 WARN Utils: Your hostname, marcelo-Aspire-A315-42G resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp4s0)
22/11/01 12:29:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/marcelo/anaconda3/envs/spark/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/marcelo/.ivy2/cache
The jars for the packages stored in: /home/marcelo/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9113368b-f954-430c-af82-96eaabadfd38;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.0 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 3673ms :: artifacts dl 22ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  

22/11/01 12:29:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 12:29:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/01 12:29:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Criação de Schema
schema = StructType([StructField('current_observation', \
                                  StructType([StructField('@version', StringType(), True),\
                                              StructField('@xmlns:xsd', StringType(), True),\
                                              StructField('@xmlns:xsi', StringType(), True),\
                                              StructField('@xsi:noNamespaceSchemaLocation', StringType(), True),\
                                              StructField('copyright_url', StringType(), True),\
                                              StructField('credit', StringType(), True),\
                                              StructField('credit_URL', StringType(), True),\
                                              StructField('dewpoint_c', StringType(), True),\
                                              StructField('dewpoint_f', StringType(), True),\
                                              StructField('dewpoint_string', StringType(), True),\
                                              StructField('disclaimer_url', StringType(), True),\
                                              StructField('icon_url_base', StringType(), True),\
                                              StructField('icon_url_name', StringType(), True),\
                                              StructField('image', StructType([\
                                                                               StructField('link', StringType(), True),\
                                                                               StructField('title', StringType(), True),\
                                                                               StructField('url', StringType(), True)]),True),\
                                              StructField('latitude', StringType(), True),\
                                              StructField('location', StringType(), True),\
                                              StructField('longitude', StringType(), True),\
                                              StructField('ob_url', StringType(), True),\
                                              StructField('observation_time', StringType(), True),\
                                              StructField('observation_time_rfc822', StringType(), True),\
                                              StructField('pressure_in', StringType(), True),\
                                              StructField('pressure_mb', StringType(), True),\
                                              StructField('pressure_string', StringType(), True),\
                                              StructField('privacy_policy_url', StringType(), True),\
                                              StructField('relative_humidity', StringType(), True),\
                                              StructField('station_id', StringType(), True),\
                                              StructField('suggested_pickup', StringType(), True),\
                                              StructField('suggested_pickup_period', StringType(), True),\
                                              StructField('temp_c', StringType(), True),\
                                              StructField('temp_f', StringType(), True),\
                                              StructField('temperature_string', StringType(), True),\
                                              StructField('two_day_history_url', StringType(), True),\
                                              StructField('visibility_mi', StringType(), True),\
                                              StructField('weather', StringType(), True),\
                                              StructField('wind_degrees', StringType(), True),\
                                              StructField('wind_dir', StringType(), True),\
                                              StructField('wind_kt', StringType(), True),\
                                              StructField('wind_mph', StringType(), True),\
                                              StructField('wind_string', StringType(), True),\
                                              StructField('windchill_c', StringType(), True),\
                                              StructField('windchill_f', StringType(), True),\
                                              StructField('windchill_string', StringType(), True)]), True)])

In [3]:
# Criação do Readstream, definindo seu Schema e path
df_json = spark.readStream.format("json")\
.schema(schema)\
.load('/home/marcelo/BULK/desafio_streamSpark/files/*json')

In [4]:
# Selecionando apenas as informações que serão salvas no MongoDB
select = df_json.selectExpr("CAST (current_observation.location AS STRING) AS Localizacao",
                        "CAST (current_observation.observation_time_rfc822 AS STRING) AS Data_Hora",
                        "CAST (current_observation.temp_c AS DOUBLE) AS Temperatura",
                        "CAST (current_observation.pressure_in AS DOUBLE) AS Pressao",
                        "CAST (current_observation.weather AS STRING) AS Clima")

In [5]:
# Definindo diretório do checkpoint e o deletando, caso exista
checkpoint = '/home/marcelo/BULK/desafio_streamSpark/checkpoint'
if os.path.exists(checkpoint):
    shutil.rmtree(checkpoint)

In [ ]:
# Salvando o Stream dos Dados baixados pela API python, no banco de dados MongoDB
select.writeStream\
    .format('mongodb')\
    .option('spark.mongodb.connection.uri', 'mongodb://127.0.0.1:27017/')\
    .option('spark.mongodb.database', 'marcelo')\
    .option('spark.mongodb.collection', 'weather')\
    .option("checkpointLocation", checkpoint)\
    .outputMode("append").start().awaitTermination()

22/11/01 12:29:12 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 12:29:12 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 12:29:12 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


22/11/01 13:28:54 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 13:28:54 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 13:28:54 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 14:28:55 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 14:28:55 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 14:28:55 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 15:28:57 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/01 15:28:57 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


22/11/02 11:29:28 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/02 11:29:28 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
22/11/02 11:29:28 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
